# Workbench Client

In [ ]:
from ansys.api.workbench.v0 import workbench_client as wc
from ansys.api.workbench.v0 import launch_workbench as lw
from ansys.mechanical.core import launch_mechanical as lm
import os
import json
import pyvista as pv

host = "localhost"
port = 55555
workdir = "C:/Users/fli/demo"

In [ ]:
# launch Workbench service on the remote host and retrieve the port

# username = "your_username"
# password = "your_password"
# launcher = lw.LaunchWorkbench(host, username, password)

launcher = lw.LaunchWorkbench()
port = launcher.launch()

In [ ]:
# create a local client and connect it to the server
client = wc.WorkbenchClient(workdir, host, port)
client.connect()

In [ ]:
# upload a couple of input files
client.upload_file_from_example_repo("2pipes.agdb", "2pipes")
client.upload_file("solve.py")

In [ ]:
# run a Workbench script to define the project and load geometry
output = client.run_script_file('geometry.wbjn')
print(output)

In [ ]:
# start PyMechanical service then create a local PyMechanical client
mech_port = client.run_script_file('start_pymech.wbjn')
print("mechanical grpc started on port " + str(mech_port))
mechanical = lm(start_instance=False, ip=host, port=mech_port)
print(mechanical.project_directory)

In [ ]:
# run a Mechanical python script via PyMechanical to mesh/solve the model
with open (os.path.join(workdir, "solve.py")) as sf:
    mech_script = sf.read()
mech_output = mechanical.run_python_script(mech_script)
print(mech_output)

In [ ]:
# download a couple of output files
client.download_file("solve.out")
client.download_file("deformation.png")

In [ ]:
# display the content of the solver output
with open(os.path.join(workdir, "solve.out"), "r") as f:
    print(f.read())

In [ ]:
# plot the deformation result generated by the solver
pl = pv.Plotter()
pl.add_background_image(os.path.join(workdir, "deformation.png"))
pl.show(jupyter_backend='static')

In [ ]:
# close out the client
client.close()

In [ ]:
# shutdown the Workbench service
launcher.shutdown()